In [1]:
import pandas as pd
import os
import numpy as np
import datetime

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Q2_rewards_trans_per_id_and_AOV_JL_20190710'

In [2]:
def recurisve_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

all_files_2019_daily=list(recurisve_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_files_2019_daily=[x for x in all_files_2019_daily if (".txt" in x) & ("aily" in x) ]
df_2019_daily=pd.DataFrame({"file_path":all_files_2019_daily})
df_2019_daily['week_end_dt']=df_2019_daily['file_path'].apply(lambda x: x.split("/2019_by_weeks/MediaStorm_")[1][:10])

df_2019_daily_Q2=df_2019_daily[(df_2019_daily['week_end_dt']>="2019-05-05")]
print("df_2019_daily_Q2['week_end_dt'].min(): "+str(df_2019_daily_Q2['week_end_dt'].min()))
print("df_2019_daily_Q2['week_end_dt'].max(): "+str(df_2019_daily_Q2['week_end_dt'].max()))
                        

df_2019_daily_Q2['week_end_dt'].min(): 2019-05-11
df_2019_daily_Q2['week_end_dt'].max(): 2019-07-06


In [3]:
df_Q2_rewards_trans_store=pd.DataFrame()

for file in df_2019_daily_Q2['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df[pd.notnull(df['customer_id_hashed'])]
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q2_rewards_trans_store=df_Q2_rewards_trans_store.append(df)
    
    

In [4]:
df_Q2_rewards_trans_store['store_type']=np.where(df_Q2_rewards_trans_store['location_id']=="6990","Online","Instore")
df_Q2_rewards_trans_store.head(5)

,customer_id_hashed,location_id,transaction_dt,sales,trans,store_type
0,000008374518306e87838f387eedc47d49a447707dc6da...,5235,2019-05-05,17.82,1,Instore
1,00002227290330cf6112d04470e512057ba4b50ab64c9f...,4550,2019-05-11,18.80,1,Instore
2,00002337f37bbdcc9a5338939fb5cb662a5c8691e755e8...,5373,2019-05-10,26.00,1,Instore
3,0000296fdb99c4476c91f1fbe9fe38d581e17cd1a3aeb0...,5135,2019-05-10,24.88,1,Instore
4,0000492bea62f21d029daecc6ef6f45d722be19d626ca6...,4619,2019-05-07,14.90,1,Instore


In [5]:
df_Q2_rewards_trans_type=df_Q2_rewards_trans_store.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q2_rewards_trans_type.head(5)

,customer_id_hashed,store_type,trans,sales
0,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,Instore,1,52.50
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,1,7.45
2,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,Instore,1,15.00
3,000006e85aba854de9992437786bddfbd409ddbebeeb80...,Instore,1,31.15
4,000008374518306e87838f387eedc47d49a447707dc6da...,Instore,4,59.05


In [6]:
df_Q2_rewards_trans_type['both_type']="Both_Online_and_Instore"
df_Q2_rewards_trans_type.shape

(6819013, 5)

In [7]:
agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type=df_Q2_rewards_trans_type.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both=df_Q2_rewards_trans_type.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type['avg_trans_per_reward_id']=df_output_type['trans']/df_output_type['customer_id_hashed']
df_output_type['AOV']=df_output_type['sales']/df_output_type['trans']

df_output_both['avg_trans_per_reward_id']=df_output_both['trans']/df_output_both['customer_id_hashed']
df_output_both['AOV']=df_output_both['sales']/df_output_both['trans']


In [8]:
'''
writer=pd.ExcelWriter("./BL_Q2_overall_9_weeks_rewards_trans_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_type.to_excel(writer,"online_and_instore",index=False)
df_output_both.to_excel(writer,"overall",index=False)
writer.save()
'''

'\nwriter=pd.ExcelWriter("./BL_Q2_overall_9_weeks_rewards_trans_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")\ndf_output_type.to_excel(writer,"online_and_instore",index=False)\ndf_output_both.to_excel(writer,"overall",index=False)\nwriter.save()\n'

In [9]:
all_files_2018_daily=list(recurisve_file_gen("/home/jian/BigLots/2018_by_weeks/"))
all_files_2018_daily=[x for x in all_files_2018_daily if (".txt" in x) & ("aily" in x) ]
df_2018_daily=pd.DataFrame({"file_path":all_files_2018_daily})
df_2018_daily['week_end_dt']=df_2018_daily['file_path'].apply(lambda x: x.split("/2018_by_weeks/MediaStorm_")[1][:10])

df_2018_daily_Q2=df_2018_daily[(df_2018_daily['week_end_dt']>="2018-05-06") &\
                               (df_2018_daily['week_end_dt']<="2018-07-07")]

print("df_2018_daily_Q2['week_end_dt'].min(): "+str(df_2018_daily_Q2['week_end_dt'].min()))
print("df_2018_daily_Q2['week_end_dt'].max(): "+str(df_2018_daily_Q2['week_end_dt'].max()))
                        

df_2018_daily_Q2['week_end_dt'].min(): 2018-06-16
df_2018_daily_Q2['week_end_dt'].max(): 2018-07-07


In [10]:
all_files_2018_historical=list(recurisve_file_gen("/home/jian/BigLots/hist_daily_data_subclasslevel/"))
all_files_2018_historical=[x for x in all_files_2018_historical if (".txt" in x)]
df_2018_historical=pd.DataFrame({"file_path":all_files_2018_historical})
df_2018_historical['week_end_dt']=df_2018_historical['file_path'].apply(lambda x: x.split("lySales_week_ending_")[1][:10])

df_2018_historical=df_2018_historical[(df_2018_historical['week_end_dt']>="2018-05-06") &\
                               (df_2018_historical['week_end_dt']<="2018-07-07")]

print("df_2018_historical['week_end_dt'].min(): "+str(df_2018_historical['week_end_dt'].min()))
print("df_2018_historical['week_end_dt'].max(): "+str(df_2018_historical['week_end_dt'].max()))
                        

df_2018_historical['week_end_dt'].min(): 2018-05-12
df_2018_historical['week_end_dt'].max(): 2018-06-09


In [11]:
all_2018_subclass_files=df_2018_historical.append(df_2018_daily_Q2)
all_2018_subclass_files.shape

(9, 2)

In [12]:
df_Q2_rewards_trans_store_2018=pd.DataFrame()

for file in all_2018_subclass_files['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[pd.notnull(df['customer_id_hashed'])]
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q2_rewards_trans_store_2018=df_Q2_rewards_trans_store_2018.append(df)


In [13]:
df_Q2_rewards_trans_store_2018['store_type']=np.where(df_Q2_rewards_trans_store_2018['location_id']=="6990","Online","Instore")
df_Q2_rewards_trans_store_2018.head(5)

,customer_id_hashed,location_id,transaction_dt,sales,trans,store_type
0,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,4133,2018-06-07,20.3,1,Instore
1,00002227290330cf6112d04470e512057ba4b50ab64c9f...,4148,2018-06-06,9.9,1,Instore
2,00003b888de64a8e334f9053611ea5fbfb0a27e168a5be...,1408,2018-06-06,28.5,1,Instore
3,00004962baa9bd312e5d13e1aebd66dd7b96bd28cd367d...,5194,2018-06-04,6.2,1,Instore
4,000058328a5153892dad9a2c180605607e24e7d2e0c09b...,4447,2018-06-05,25.4,1,Instore


In [14]:
df_Q2_rewards_trans_type_2018=df_Q2_rewards_trans_store_2018.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q2_rewards_trans_type_2018.head(5)

,customer_id_hashed,store_type,trans,sales
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Instore,5,117.99
1,000006aa196c7510ca3b4617e7e52d9527e0190e9152fb...,Instore,1,20.30
2,000008374518306e87838f387eedc47d49a447707dc6da...,Instore,2,22.77
3,000009e0ff428cd05649d579c6b188ad41635b662f79f5...,Instore,2,103.96
4,00000b02775629a353b0732eead2135aac6bb0502f10db...,Instore,1,172.30


In [15]:
df_Q2_rewards_trans_type_2018['both_type']="Both_Online_and_Instore"
df_Q2_rewards_trans_type_2018.shape

(6129670, 5)

In [16]:
agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type_2018=df_Q2_rewards_trans_type_2018.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both_2018=df_Q2_rewards_trans_type_2018.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type_2018['avg_trans_per_reward_id']=df_output_type_2018['trans']/df_output_type_2018['customer_id_hashed']
df_output_type_2018['AOV']=df_output_type_2018['sales']/df_output_type_2018['trans']

df_output_both_2018['avg_trans_per_reward_id']=df_output_both_2018['trans']/df_output_both_2018['customer_id_hashed']
df_output_both_2018['AOV']=df_output_both_2018['sales']/df_output_both_2018['trans']


In [17]:
df_output_both_2018

,both_type,trans,sales,customer_id_hashed,avg_trans_per_reward_id,AOV
0,Both_Online_and_Instore,11874342,4.757298e+08,6116840,1.941254,40.063675


In [18]:
df_output_type_2018

,store_type,trans,sales,customer_id_hashed,avg_trans_per_reward_id,AOV
0,Instore,11837007,4.722147e+08,6103861,1.939265,39.893085
1,Online,37335,3.515057e+06,25809,1.446588,94.149104


In [19]:
all_files_2018_daily_later=list(recurisve_file_gen("/home/jian/BigLots/2018_by_weeks/"))
all_files_2018_daily_later=[x for x in all_files_2018_daily if (".txt" in x) & ("aily" in x) ]
df_2018_daily_later=pd.DataFrame({"file_path":all_files_2018_daily_later})
df_2018_daily_later['week_end_dt']=df_2018_daily_later['file_path'].apply(lambda x: x.split("/2018_by_weeks/MediaStorm_")[1][:10])

df_2018_daily_Q2_later=df_2018_daily[(df_2018_daily_later['week_end_dt']>"2018-07-07") &\
                               (df_2018_daily_later['week_end_dt']<="2018-08-04")]

print("df_2018_daily_Q2_later['week_end_dt'].min(): "+str(df_2018_daily_Q2_later['week_end_dt'].min()))
print("df_2018_daily_Q2_later['week_end_dt'].max(): "+str(df_2018_daily_Q2_later['week_end_dt'].max()))
                        

df_2018_daily_Q2_later['week_end_dt'].min(): 2018-07-14
df_2018_daily_Q2_later['week_end_dt'].max(): 2018-08-04


In [20]:
# The whole Q2 2018 with 13 weeks
for file in df_2018_daily_Q2_later['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[pd.notnull(df['customer_id_hashed'])]
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q2_rewards_trans_store_2018=df_Q2_rewards_trans_store_2018.append(df)


In [25]:
df_Q2_rewards_trans_store_2018.head(2)

,customer_id_hashed,location_id,trans,sales,both_type
0,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,1852,2,449.46,Both_Online_and_Instore
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,5116,7,117.99,Both_Online_and_Instore


In [26]:
df_Q2_rewards_trans_store_2018['store_type']=np.where(df_Q2_rewards_trans_store_2018['location_id']=="6990","Online","Instore")
df_Q2_rewards_trans_type_2018=df_Q2_rewards_trans_store_2018.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q2_rewards_trans_type_2018['both_type']="Both_Online_and_Instore"


agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type_2018_whole=df_Q2_rewards_trans_type_2018.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both_2018_whole=df_Q2_rewards_trans_type_2018.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type_2018_whole['avg_trans_per_reward_id']=df_output_type_2018_whole['trans']/df_output_type_2018_whole['customer_id_hashed']
df_output_type_2018_whole['AOV']=df_output_type_2018_whole['sales']/df_output_type_2018_whole['trans']

df_output_both_2018_whole['avg_trans_per_reward_id']=df_output_both_2018_whole['trans']/df_output_both_2018_whole['customer_id_hashed']
df_output_both_2018_whole['AOV']=df_output_both_2018_whole['sales']/df_output_both_2018_whole['trans']


In [27]:
writer=pd.ExcelWriter("./BL_Q2_overall_9_weeks_18_19_rewards_trans_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_type.to_excel(writer,"online_and_instore_19_9W",index=False)
df_output_both.to_excel(writer,"overall_19_9W",index=False)

df_output_type_2018.to_excel(writer,"online_and_instore_18_9W",index=False)
df_output_both_2018.to_excel(writer,"overall_18_9W",index=False)

df_output_type_2018_whole.to_excel(writer,"online_and_instore_18_13W",index=False)
df_output_both_2018_whole.to_excel(writer,"overall_18_13W",index=False)

writer.save()

In [28]:
df_Q2_rewards_trans_store_2018.head(2)

,customer_id_hashed,location_id,trans,sales,both_type,store_type
0,0000034fec3c2cbbf7a2b38fbf80d84abc72d8bed269c4...,1852,2,449.46,Both_Online_and_Instore,Instore
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,5116,7,117.99,Both_Online_and_Instore,Instore
